<p align="center">
  <img src="header.png" width="100%">
</p>


<div style="text-align: center;">
    <strong style="display: block; margin-bottom: 10px;">Group P</strong> 
    <table style="margin: 0 auto; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid white; padding: 8px;">Name</th>
            <th style="border: 1px solid white; padding: 8px;">Student ID</th>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Beatriz Monteiro</td>
            <td style="border: 1px solid white; padding: 8px;">20240591</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Catarina Nunes</td>
            <td style="border: 1px solid white; padding: 8px;">20230083</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Margarida Raposo</td>
            <td style="border: 1px solid white; padding: 8px;">20241020</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Teresa Menezes</td>
            <td style="border: 1px solid white; padding: 8px;">20240333</td>
        </tr>
    </table>
</div>

### 🔗 Table of Contents <a id='table-of-contents'></a>
1. [Introduction](#introduction)  
2. [Macro's Prediction](#business-understanding)  
3. [Conclusion](#conclusion)

---

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller
from scipy.stats import norm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [20]:
df_market_sales = pd.read_csv("df_market_sales.csv")
df_market_sales.head() 

,P1,P3,P4,P5,P6,P8,P9,P11,P12,P13,...,PRO28250_org,PRO28276_org,PRO27840_org,PRO271000_org,PRO27756_org,PRO27826_org,PRO27380_org,PRO27392_org,PRO27250_org,PRO27276_org
0,3.577403e+07,8.016661e+06,394180.84379,2.476570e+06,365908.51560,580778.26534,3190.34612,1.012112e+06,28428.15303,27417.10510,...,114.720810,127.461136,112.853256,129.325775,112.970843,118.670791,93.001511,112.376774,97.849541,118.298233
1,5.063649e+06,1.168506e+07,365752.58370,8.859035e+06,465951.25560,518398.37845,1847.76150,1.870362e+06,1053.95000,67157.30000,...,104.873100,132.987915,113.145294,128.236176,114.736013,120.467019,84.133400,111.907535,91.155960,117.163727
2,3.732127e+07,8.605806e+06,423649.44560,6.843295e+06,984480.59965,267418.34935,0.00000,1.207730e+06,17612.39100,15420.35230,...,104.974617,137.363281,111.823624,117.043549,116.501182,105.378705,64.881248,112.524242,78.033028,89.626122
3,2.709040e+07,1.045908e+07,473037.88076,8.043841e+06,585099.99570,372627.94655,1452.79900,2.165415e+06,20989.86800,26572.72933,...,95.228432,104.432930,109.499725,111.522202,110.354610,107.174933,81.556343,95.957352,81.908257,105.406097
4,3.413209e+07,9.974609e+06,608251.19580,6.748545e+06,531738.80712,361474.53417,3172.82868,5.988579e+05,33564.65994,32018.49603,...,100.101524,112.838226,110.835655,112.666725,104.208038,110.647640,83.451233,107.309956,80.322929,103.652771


In [21]:
sales_final = pd.read_csv("sales_final.csv")
sales_final.head()

,P1,P3,P4,P5,P6,P8,P9,P11,P12,P13,...,P11%Change,P12%Change,P13%Change,P14%Change,P16%Change,P20%Change,P36%Change,Sales_EUR%Change,Consumer Price,Sales_CPI
0,3.577403e+07,8.016661e+06,394180.84379,2.476570e+06,365908.51560,580778.26534,3190.34612,1.012112e+06,28428.15303,27417.10510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.1,4.902613e+07
1,5.063649e+06,1.168506e+07,365752.58370,8.859035e+06,465951.25560,518398.37845,1847.76150,1.870362e+06,1053.95000,67157.30000,...,0.859237,-0.962700,1.464388,1.968804,3.134776,0.270647,2.338833,-0.378215,98.5,3.029917e+07
2,3.732127e+07,8.605806e+06,423649.44560,6.843295e+06,984480.59965,267418.34935,0.00000,1.207730e+06,17612.39100,15420.35230,...,-0.354280,15.710841,-0.770385,0.000000,2.456881,-0.841561,-0.355871,0.993203,98.5,6.039239e+07
3,2.709040e+07,1.045908e+07,473037.88076,8.043841e+06,585099.99570,372627.94655,1452.79900,2.165415e+06,20989.86800,26572.72933,...,0.807644,0.201526,0.737335,-0.901553,-0.802005,1.610396,0.109151,-0.162316,97.7,5.017887e+07
4,3.413209e+07,9.974609e+06,608251.19580,6.748545e+06,531738.80712,361474.53417,3172.82868,5.988579e+05,33564.65994,32018.49603,...,-0.724572,0.592568,0.200025,10.383005,-0.726642,-0.741180,-0.478701,0.057959,98.1,5.330453e+07


In [22]:
df_market_filtered = pd.read_csv("df_market_filtered.csv")
df_market_filtered.head()

,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_PRO250,MAB_ELE_SHP250,MAB_ELE_PRO276,MAB_ELE_SHP276,MAB_ELE_PRO380,MAB_ELE_SHP380,MAB_ELE_PRO392,MAB_ELE_SHP392,...,PRO28250_org,PRO28276_org,PRO27840_org,PRO271000_org,PRO27756_org,PRO27826_org,PRO27380_org,PRO27392_org,PRO27250_org,PRO27276_org
0,211.955755,211.955755,108.280608,122.451734,124.227879,137.741953,118.350514,122.456894,125.209957,124.793250,...,114.720810,127.461136,112.853256,129.325775,112.970843,118.670791,93.001511,112.376774,97.849541,118.298233
1,220.519655,220.519655,99.636911,115.958210,127.404132,142.732193,107.719260,120.132032,122.624695,123.289888,...,104.873100,132.987915,113.145294,128.236176,114.736013,120.467019,84.133400,111.907535,91.155960,117.163727
2,241.846854,241.846854,94.690312,115.128469,120.518565,141.407661,88.783181,131.936099,122.991956,124.508413,...,104.974617,137.363281,111.823624,117.043549,116.501182,105.378705,64.881248,112.524242,78.033028,89.626122
3,175.668147,175.668147,90.143775,108.325154,104.776326,118.703828,99.275113,92.418842,105.514024,106.303946,...,95.228432,104.432930,109.499725,111.522202,110.354610,107.174933,81.556343,95.957352,81.908257,105.406097
4,175.668147,175.668147,92.551521,108.944656,109.597012,122.686997,104.586684,103.214212,114.466647,114.996302,...,100.101524,112.838226,110.835655,112.666725,104.208038,110.647640,83.451233,107.309956,80.322929,103.652771


### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Prediction for the Macro Features used <a id='business-understanding'></a>

##### Click [here](#table-of-contents) ⬆️ to return to the Index.
---

In [13]:
# Define the function to classify variables
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    
    # Remove NaN values for testing
    clean_series = series.dropna()

    # Check for normality
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 means normal
    else:
        is_normal = False  # Not enough data to test normality

    # Check for stationarity
    if len(clean_series) > 3:
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 means stationary
    else:
        is_stationary = False  # Not enough data

    return is_normal, is_stationary

# Function to automatically fill missing values
def auto_impute_missing_values(df_train, df_test):
    """Automatically selects the best imputation method for each missing variable."""
    
    # Identify missing columns in test set
    missing_columns = df_test.columns[df_test.isnull().any()]
    
    # Iterate through missing columns
    for col in missing_columns:
        print(f"Processing: {col}")

        series = df_train[col]  # Use train data for imputation
        is_normal, is_stationary = classify_variable(series)

        if is_normal:
            # Case 1: Normally distributed → Sample from normal distribution
            print(f" - {col} is normal → Using Mean & Std Sampling")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            # Case 2: Stationary but non-normal → Simple Exponential Smoothing
            print(f" - {col} is stationary → Using Simple Exponential Smoothing")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        elif not is_stationary:
            # Case 3: Non-Stationary → ARIMA
            print(f" - {col} is non-stationary → Using ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))  # (p,d,q) chosen based on domain knowledge
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        else:
            # Case 4: If nothing works → Use XGBoost Regression
            print(f" - {col} is complex → Using XGBoost Regression")
            train_data = df_train.dropna(subset=[col])  # Drop missing values for training
            X_train = train_data.drop(columns=[col])  # Exclude target column
            y_train = train_data[col]  # Target column

            X_test = df_test.loc[df_test[col].isnull(), X_train.columns]  # Only missing values

            model = XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Assign predictions
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Example usage
df_train = remerged_data[1]  # Use remerged train data
df_test = test_1.copy()  # Copy test set

# Apply automatic imputation
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Check results
print(df_test_filled.head())

NameError: name 'remerged_data' is not defined